In [1]:
import sevenbridges as sbg
import yaml, json

In [2]:
# create the api object 
config_yaml = '../data/config.yaml'
base_url = 'https://cavatica-api.sbgenomics.com/v2/'

with open(config_yaml, 'r') as y:
    config = yaml.load(y)

api = sbg.Api(url=base_url, token=config['token'])

## upload cwl
Require to get the whole workflow into one single JSON in advance

`rabix --resolve-app workflow.cwl > workflow.json`

In [3]:
# app id
# project = 'yuankun/kf-workflow-dev'
project = 'yuankun/kf-workflow-test-run'
app = 'kfdrc_alignment_pipeline_cavatica_api'
app_id = '/'.join((project, app))

# load cwl
cwl = '../data/kfdrc_alignment_pipleine_cavatica_api.json'
with open(cwl, 'r') as j:
    raw = json.load(j)

# get revision
try:
    app = api.apps.get(id=app_id)
    app_id = app_id + '/' + str(app.revision + 1)
except:
    pass

# install app
api.apps.install_app(app_id, raw)

<App: id=yuankun/kf-workflow-test-run/kfdrc_alignment_pipeline_cavatica_api rev=0>

## run task

In [27]:
references = api.files.query(project='yuankun/kf-reference').all()
input_id = '../data/reference-inputs-cavatica-2.json'
with open(input_id, 'r') as j:
    input_id = json.load(j)
   
inputs = {}
inputs['knownsites'] = []

for k in input_id.keys():
    if (k=='knownsites'):
        for i in input_id[k]:
            inputs[k].append(api.files.get(id=i))
    else:
        inputs[k] = api.files.get(id=input_id[k])

In [28]:
# run single task
name = 'api-task-run ref-from-this-project'
project = 'yuankun/kf-workflow-test-run'
app = project + '/kfdrc_alignment_pipeline_cavatica_api'

inputs['input_bam'] = api.files.get(id='5a466edc4f0c197e3614e33d') # sample.bam

try:
    task = api.tasks.create(
        name=name,
        project=project,
        app=app,
        inputs=inputs,
        run=True)
except:
    print 'I was unable to run the task.'


In [35]:
# batch tasks
name = 'api-task-batch-run-3'
project = 'yuankun/kf-workflow-test-run'
app = project + '/kfdrc_alignment_pipeline_cavatica_api'
            
inputs['input_bam'] = api.files.query(project=project, tags='input_bam')
batch_input = 'input_bam'
batch_by = {'type': 'item'}

try:
    task = api.tasks.create(
        name=name,
        project=project,
        app=app,
        batch_input=batch_input,
        batch_by=batch_by,
        inputs=inputs,
        run=True)
except:
    print 'I was unable to run the task.'